In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
df = pd.read_excel(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/sports/extractions_denombrement.ods"
)

In [ ]:
df.shape

In [ ]:
fields = {
    "entrée unitaire": {
        "categorie": "entrée unitaire",
        "champ_pu": "Pu entrée unitaire",
        "openfisca_input_variable": "strasbourg_piscine_entree_unitaire",
        "openfisca_output_variable": "strasbourg_piscine_prix_entree_unitaire",
    },
    "10 entrées": {
        "categorie": "10 entrées",
        "champ_pu": "Pu 10 entrées",
        "openfisca_input_variable": "strasbourg_piscine_10_entrees",
        "openfisca_output_variable": "strasbourg_piscine_prix_10_entrees",
    },
    "abo_annuel": {
        "categorie": "abo_annuel",
        "champ_pu": "pu abo_annuel",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_annuel",
        "openfisca_output_variable": "strasbourg_prix_piscine_abonnement_annuel",
    },
    "abo_annuel_ce": {
        "categorie": "abo_annuel_ce",
        "champ_pu": "pu abo_annuel_ce",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_ce",
        "openfisca_output_variable": "strasbourg_prix_piscine_abonnement_ce",
    },
    "5 entrées ce": {
        "categorie": "5 entrées ce",
        "champ_pu": "pu 5 entrées ce",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_ce",
        "openfisca_output_variable": "strasbourg_prix_piscine_5_entrees_ce",
    },
    "cycle": {
        "categorie": "cycle",
        "champ_pu": "pu cycle",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_ce",
        "openfisca_output_variable": "strasbourg_prix_piscine_cycle",
    },
    "stage été": {
        "categorie": "stage été",
        "champ_pu": "pu stage été",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_ce",
        "openfisca_output_variable": "strasbourg_prix_piscine_stage_ete",
    },
    "stage vacances": {
        "categorie": "stage vacances",
        "champ_pu": "pu stage vacances",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_ce",
        "openfisca_output_variable": "strasbourg_prix_piscine_stage_vacances",
    },
    "stage 5 séances": {
        "categorie": "stage 5 séances",
        "champ_pu": "pu stage 5 séances",
        "openfisca_input_variable": "strasbourg_piscine_abonnement_ce",
        "openfisca_output_variable": "strasbourg_prix_piscine_stage_5_seances",
    },
    "patinoire entree unitaire": {
        "categorie": "patinoire entree unitaire",
        "champ_pu": "pu patinoire entree unitaire",
        "openfisca_input_variable": "strasbourg_patinoire_entree_unitaire",
        "openfisca_output_variable": "strasbourg_prix_patinoire_entree_unitaire",
    },
    "patinoire 10 entrees": {
        "categorie": "patinoire 10 entrees",
        "champ_pu": "pu patinoire 10 entrees",
        "openfisca_input_variable": "strasbourg_patinoire_10_entrees",
        "openfisca_output_variable": "strasbourg_prix_patinoire_10_entrees",
    },
    "patinoire 5 entrees ce": {
        "categorie": "patinoire 5 entrees ce",
        "champ_pu": "pu patinoire 5 entrees ce",
        "openfisca_input_variable": "strasbourg_patinoire_5_entrees_ce",
        "openfisca_output_variable": "strasbourg_prix_patinoire_5_entrees_ce",
    },
}

In [ ]:
done_indexes = {}
for v in fields:
    done_indexes[v] = ~df[v].isna()

In [ ]:
done = pd.DataFrame(data=done_indexes)

In [ ]:
df[(~done.any(axis=1)) * df.exclu.isna()]  # *(df.type == "Piscine")]

In [ ]:
def compute(categorie):
    champ_pu = fields[categorie]["champ_pu"]
    openfisca_input_variable = fields[categorie]["openfisca_input_variable"]
    openfisca_output_variable = fields[categorie]["openfisca_output_variable"]

    product_df = df[~df[categorie].isna()]
    individu_df = pd.DataFrame(
        {
            "famille_id": list(range(len(product_df))),
            "agerule": product_df.age,
            "taux_incapacite": np.where(
                product_df.qfrule.str.contains("HANDICAP"), 0.8, 0
            ),
            openfisca_input_variable: product_df.quantité,
        }
    )
    determine_age(individu_df)

    famille_df = pd.DataFrame(
        {
            "qfrule": product_df.qfrule,
        }
    )
    determine_qf(famille_df, qfrules_constant)

    menage_df = pd.DataFrame({})
    foyerfiscaux_df = pd.DataFrame({})

    individu_df["famille_role_index"] = 0
    individu_df["foyer_fiscal_id"] = individu_df.famille_id
    individu_df["foyer_fiscal_role_index"] = 0
    individu_df["menage_id"] = individu_df.famille_id
    individu_df["menage_role_index"] = 0

    data = dict(
        input_data_frame_by_entity=dict(
            individu=individu_df,
            famille=famille_df,
            menage=menage_df,
            foyer_fiscal=foyerfiscaux_df,
        )
    )

    scenario = StrasbourgSurveyScenario(base, data=data)

    prix = scenario.simulation.calculate(openfisca_output_variable, base_period)
    res_prix = pd.DataFrame(
        data={
            "prestation": product_df.prestation,
            "qf_caf": famille_df.qf_caf,
            "age": individu_df.age,
            "res": (prix - product_df[champ_pu]).abs() < 0.001,
            "prix_input": product_df[champ_pu],
            "prix_output": prix,
        }
    )
    return res_prix

In [ ]:
len(fields)

In [ ]:
for v in fields:
    print(v)
    res = compute(v)
    display(res[~res.res])
    assert res[~res.res].shape[0] == 0

In [ ]:
def build_sample(product_df, sample_count):
    count = sum(product_df.quantité)
    sample_ids = np.repeat(list(range(sample_count)), count)
    sample_qfrule = np.tile(
        np.repeat(product_df.qfrule, product_df.quantité), sample_count
    )
    sample_individu_df = pd.DataFrame(
        {
            "sample_id": sample_ids,
            "famille_id": list(range(count * sample_count)),
            "agerule": np.tile(
                np.repeat(product_df.age, product_df.quantité), sample_count
            ),
            "taux_incapacite": np.tile(
                np.repeat(
                    np.where(product_df.qfrule.str.contains("HANDICAP"), 0.8, 0),
                    product_df.quantité,
                ),
                sample_count,
            ),
        }
    )

    determine_age(sample_individu_df)

    sample_famille_df = pd.DataFrame(
        {
            "qfrule": sample_qfrule,
        }
    )
    determine_qf(sample_famille_df, qfrules_constant)

    sample_menage_df = pd.DataFrame({})
    sample_foyerfiscaux_df = pd.DataFrame({})

    sample_individu_df["famille_role_index"] = 0
    sample_individu_df["foyer_fiscal_id"] = sample_individu_df.famille_id
    sample_individu_df["foyer_fiscal_role_index"] = 0
    sample_individu_df["menage_id"] = sample_individu_df.famille_id
    sample_individu_df["menage_role_index"] = 0

    sample_data = dict(
        input_data_frame_by_entity=dict(
            individu=sample_individu_df,
            famille=sample_famille_df,
            menage=sample_menage_df,
            foyer_fiscal=sample_foyerfiscaux_df,
        )
    )

    return sample_data, sample_ids


def compute_sample(categorie, sample_count=2):
    champ_pu = fields[categorie]["champ_pu"]
    openfisca_input_variable = fields[categorie]["openfisca_input_variable"]
    openfisca_output_variable = fields[categorie]["openfisca_output_variable"]

    product_df = df[~df[categorie].isna()]

    sample_data, sample_ids = build_sample(product_df, sample_count)
    prix = np.tile(np.repeat(product_df[champ_pu], product_df.quantité), sample_count)

    sample_data["input_data_frame_by_entity"]["individu"][openfisca_input_variable] = 1

    sample_scenario = StrasbourgSurveyScenario(base, data=sample_data)
    sample_prix = sample_scenario.simulation.calculate(
        openfisca_output_variable, base_period
    )

    sample_res_df = pd.DataFrame(
        data={
            "prestation": np.tile(
                np.repeat(product_df.prestation, product_df.quantité), sample_count
            ),
            "categorie": np.tile(
                np.repeat(product_df.categorie, product_df.quantité), sample_count
            ),
            "index": np.tile(
                np.repeat(product_df.index, product_df.quantité), sample_count
            ),
            "sample_id": sample_ids,
            "input_prix": prix,
            "output_prix": sample_prix,
        }
    )
    return sample_res_df

In [ ]:
categorie = "entrée unitaire"
v = compute_sample(categorie)
v

In [ ]:
df_interest = df[df.categorie.isin(fields.keys())]

In [ ]:
sample_data, sample_ids = build_sample(df[df.categorie.isin(fields.keys())], 2)

In [ ]:
sample_scenario = StrasbourgSurveyScenario(base, data=sample_data)

In [ ]:
results = pd.DataFrame(
    data={
        v: sample_scenario.simulation.calculate(
            fields[v]["openfisca_output_variable"], base_period
        )
        for v in fields
    }
)
results

In [ ]:
results[df_interest.categorie]

In [ ]:
v.groupby(by=["sample_id", "categorie"]).sum()

In [ ]:
df[df.categorie == "patinoire entree unitaire"]["euros"].sum()

In [ ]:
sample_res_df = v

In [ ]:
sample_sum = sample_res_df.groupby(["sample_id"]).sum()
sample_sum

In [ ]:
sample_sum